# Pysically informed neural networks - The Two Element Windkessel model

Here we will implement the two element Windkessel in a Pysically Informed Neural Network. The model is affected by the parameters R and C, also known as total vascular resitance and total arterial compliance. The equation describing the model is the following:

\begin{equation*}
\frac{\mathrm{d}p}{\mathrm{dt}} = \frac{Q_{\mathrm{in}}}{C} - \frac{p}{R C} 
\end{equation*}

The compliant vessels of the systemic circulation are filled and stressed by blood ejected in systole from the heart. During diastole the elatic recoil of the vessel walls smoothes the blood flow in the vessels and keeps the circulation in motion even when the heart is not ejecting blood. This effect is named after an air chamber used to produce smooth water flows for fire fighting in the past. See the Figure below for a simple illustration. 

<img src = "http://folk.ntnu.no/nikolalb/ML_ws/Windkessel_effect.svg" width=600>

The model computes the blood pressure waveform $P_{ao}(t)$ (used interchangeably with $p$)* according to the imposed inflow $Q_{\mathrm{in}}$, and the specified parameters $R$ and $C$. If we use this model to produce data and then build a PINN loss function implementing the equation above, taking the input flow and a few points sampled from the model produced pressure function. 

In the code below, $f$ refers to the sum:

\begin{equation*}
f = \frac{\mathrm{d}p}{\mathrm{dt}} - \frac{Q_{\mathrm{in}}}{C} + \frac{p}{R C}
\end{equation*}

which ideally should be equal to zero, and must be minimized to obey the physics of the problem. $u$ refers to our variable of interest, which here is $p$ used interchangeably with $P_{ao}$. 

Problem 1:
- Tune the hyperparameters number of nodes, layers, epochs, training rate, but also the number of pressure training points, number of flow points, and try to get a good fit. Can you implement an ordinary neural network which can learn the pressure? Which can you get to perform best?

For this problem a gaussian inflow is prescribed, you can also attempt with a real input flow from the file "./Inflow/pqdata.mat", and see how this affects the results. Use the functions "import data" and "periodic_func_from_data" to achieve this.

In [ ]:
import numpy as np
import scipy.integrate as scint
import scipy.io as sio
import matplotlib.pyplot as plt
%matplotlib inline

#==================================================
# Set up a 2 element Windkessel model for production of data
#==================================================

class WK2():
    def __init__(self, pars):
        self.R_sys = pars["R_sys"]
        self.C_sa = pars["C_sa"]
    
    def gaussian_flow(self,t):
        tau = np.mod(t,1.0)
        gaussian = 450.0*np.exp(-(tau-0.2)**2 / (0.09)**2)
        return gaussian
    
    def rhs(self, t, u):
        P_ao = u[0]
        tau = np.mod(t,1.0)
        Q_lvao = self.gaussian_flow(tau)
        Q_aosv = P_ao/self.R_sys
        
        der_P_ao = (Q_lvao - Q_aosv)/self.C_sa
        der_u = [der_P_ao]
        
        return der_u

    def calc_all(self, t, u):
        P_ao = u[0]
        tau = np.mod(t,1.0)
        Q_lvao = self.gaussian_flow(tau)
        Q_aosv =  P_ao/self.R_sys
        
        all_vars = locals()
        del all_vars["self"]  
        del all_vars["u"]

        der_P_ao = (Q_lvao - Q_aosv)/self.C_sa
        der_u = [der_P_ao]

        return der_u, all_vars

def periodic_func_from_data(self,x_data, y_data, n_modes=40):
    dx = x_data[1]-x_data[0]
    N = len(self.t)
    # Normalization by /N and symmetry
    Y_fft = np.fft.fft(y_data)/N 
    Y_fft[1:] *= 2
    fft_freq = np.fft.fftfreq(N, d=dx)

    def periodic_func(x):
        y_trig = np.zeros_like(x)
        for kk in range(n_modes):
            y_trig += np.real(Y_fft[kk]*np.exp(2j*np.pi * fft_freq[kk] * x))
        return y_trig
    
    return periodic_func
    
def import_data(filepath):

    data = sio.loadmat(filepath)
    q = data["q"]
    p = data["p"]
    p=p.flatten()  # convert array to vector
    q=q.flatten()
    N = len(q)

    t= data["t"]
    t=t.flatten()
    dt = t[1]-t[0]
        
    return  p, q, t, dt, N
    
def get1D_data(N, showPlots=True):
    #Calculate data
    P_ao_0 = 100.0
    u0 = [P_ao_0]
    
    Parameters = dict(R_sys=1.5,C_sa=1.5)
    wkmod = WK2(Parameters)
    Ncycles = 20
    tmax = Ncycles*1.0 #Ncycles
    t_eval_vals = np.linspace(0.0, tmax, N*Ncycles)
    
    ################################################################
    ###################  solve_ivp routine  ########################
    ################################################################ 
    sol = scint.solve_ivp(wkmod.rhs, (0.0, tmax), u0, max_step=0.01, t_eval=t_eval_vals)
    _, all_vars = wkmod.calc_all(sol.t, sol.y)
    P_ao_fin_cyc = all_vars["P_ao"][int((Ncycles-1)*N):]
    t_fin = all_vars["t"][0:N]
    q_lvao = wkmod.gaussian_flow(t_fin)
    
    #P_np, P_q_np, integrand_q_array = calcDeltaP(t_in, integrand_q, P_in=0)
    if showPlots:
        plt.figure()
        plt.plot(t_fin, P_ao_fin_cyc)
        plt.legend(["P_ao"])
        #plt.plot(x_np, linearTapering_np(x_np, R0, Rmin, l))
        plt.xlabel("t [s]")
        plt.ylabel("P [mmHg]")
        plt.title("Pressure curve for data sampling")
        plt.show()
    
    return t_fin, P_ao_fin_cyc, q_lvao, wkmod.R_sys, wkmod.C_sa

#x_np, r_np, P_np, P_f_np, P_c_np, integrand_f_array, integrand_c_array = get1D_data(101)

In [ ]:
#====================================
# Functions for setting up the neural network
#====================================

def net_u(t):
    u = neural_net(t, weights, biases)
    return u
#===================================================

def net_f_WK(t, Q, R, C):
    
    u = net_u(t)
    
    u_t = tf.gradients(u, t)
    
    f =  u_t + u/(R*C) - Q/C
    
    return f
#===================================================

def initialize_NN(layers):        
    weights = []
    biases = []
    num_layers = len(layers) 
    for l in range(0, num_layers - 1):
        W = xavier_init(size=[layers[l], layers[l+1]])
        #W = tf.Variable(tf.random_normal([layers[l], layers[l+1]], stddev=10))
        b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
        weights.append(W)
        biases.append(b)        
    return weights, biases
#===================================================
  
def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]        
    xavier_stddev = np.sqrt(2/(in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
#===================================================

def neural_net(X, weights, biases):
    num_layers = len(weights) + 1
    
    H = 2.0*(X - lb)/(ub - lb) - 1.0
    for l in range(0, num_layers - 2):
        W = weights[l]
        b = biases[l]
        H = tf.tanh(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H, W), b)
    return Y
#===================================================

def callback(loss):
    print('Loss:', loss)

In [ ]:
import tensorflow as tf
%matplotlib inline
#===================================================
# load data
#===================================================
N = 1001 # total number of x, P values
N_train = 10 # number of training points
N_train_f = 200 # number of training points for f, i.e., input flow term and pressure term

t, P_data, Q_data, R, C = get1D_data(N, showPlots=True)

#===================================================
# set parameters for neural network and training
#===================================================
layers = [1, 3, 1]

init = tf.global_variables_initializer()
#===================================================
# turn 1D array into 2D array of shape (N, 1)
#===================================================
t = t[:, np.newaxis]
P_data = P_data[:, np.newaxis]
Q_data = Q_data[:, np.newaxis]
#===================================================
# sample random points for training
#===================================================
idx = np.random.choice(t.shape[0], N_train, replace=False)
idx_q = np.random.choice(t.shape[0], N_train_f, replace=False)
t_train = t[idx]
t_f_train = t[idx_q]#t[idx]
P_train = P_data[idx]
#t_train = t[0:1,0:1]
#q_train = P[0:1,0:1]

q_train = Q_data[idx_q]
#q_train = Q_data[idx]
#===================================================
# Set up placeholder for inputs and outputs
#===================================================
C_tf = tf.constant(C)
R_tf = tf.constant(R)

t_tf = tf.placeholder(tf.float32, shape=[None, t.shape[1]], name='t')
t_f_tf = tf.placeholder(tf.float32, shape=[None, t.shape[1]], name='t')
P_tf = tf.placeholder(tf.float32, shape=[None, P_data.shape[1]], name='P')

q_tf = tf.placeholder(tf.float32, shape=[None, Q_data.shape[1]], name='Q')
#===================================================
# initialize neural net and f functions
#===================================================
lb = t.min(0)
ub = t.max(0)
weights, biases = initialize_NN(layers)

P_pred = net_u(t_tf)
f_pred = net_f_WK(t_f_tf, q_tf, C_tf, R_tf) 


In [ ]:
#===================================================
# plot y_pred before training
#===================================================
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) # initialize variables
    P_pred_init = sess.run(P_pred, feed_dict = {t_tf:t_train})
plt.figure()
plt.plot(t.flatten(), P_data.flatten(), label = "Data")
plt.plot(t_train.flatten(), P_train.flatten(), 'o', label="Training points")
plt.plot(t_train.flatten(), P_pred_init.flatten(), label = "Init")
plt.legend()
plt.show()

In [ ]:
#===========================================================#
# train the model using PINNS and GradientDescentOptimizer  #
#===========================================================#
batch_size = N_train
loss = tf.reduce_mean(tf.square(P_tf - P_pred)) + tf.reduce_mean(tf.square(f_pred))
learning_rate = 0.002
#learning_rate_late = 0.5
epochs = 30000
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

print_every_N_batch = 1000
with tf.Session() as sess:
    sess.run(init) # initialize variables
    for epoch in range(epochs):
        avg_cost = 0
        _, c = sess.run([optimiser, loss], 
                     feed_dict={t_tf: t_train, P_tf: P_train, 
                                t_f_tf: t_f_train, q_tf:q_train})
        avg_cost += c
        if epoch % print_every_N_batch == 0:
            print("Epoch:", (epoch + 1), "cost =", "{:.6f}".format(avg_cost))

    P_result = sess.run(P_pred, feed_dict = {t_tf:t})
    plt.figure()
    plt.plot(t.flatten(), P_data.flatten())
    plt.plot(t_train.flatten(), P_train.flatten(), 'o')
    plt.plot(t.flatten(), P_result.flatten(), 'k')

In [ ]:
#===========================================================#
# train the model using PINNS and ScipyOptimizerInterface   #
#===========================================================#

optimizer = tf.contrib.opt.ScipyOptimizerInterface(loss, 
                                                   method = 'L-BFGS-B', 
                                                   options = {'maxiter': 500,
                                                              'maxfun': 50000,
                                                              'maxcor': 50,
                                                              'maxls': 50,
                                                              'ftol' : 1.0 * np.finfo(float).eps})
with tf.Session() as sess:
    sess.run(init)
    tf_dict = {t_tf: t_train, P_tf: P_train, 
                             t_f_tf: t_f_train, q_tf:q_train}
    optimizer.minimize(sess, 
                       feed_dict = tf_dict,         
                       fetches = [loss], 
                       loss_callback = callback)

    P_result = sess.run(P_pred, feed_dict = {t_tf: t})
    plt.figure()
    plt.plot(t.flatten(), P_data.flatten(),'b')
    plt.plot(t_train.flatten(), P_train.flatten(), 'yo')
    plt.plot(t.flatten(), P_result.flatten(), 'r')